In [22]:
import pandas as pd

In [23]:
df_amazon = pd.read_table("amazon_cells_labelled.txt")
df_imdb = pd.read_table("imdb_labelled.txt")
df_yelp = pd.read_table("yelp_labelled.txt")


In [24]:
# 1.Pre processing
#coverting/concatinating all our data into one single data, called frames
frames = [df_amazon, df_imdb, df_yelp]

In [25]:
#checks the first coloums 
df_yelp.columns

Index(['Wow... Loved this place.', '1'], dtype='object')

In [26]:
#renaming coloumn heading
for colname in frames:
    colname.columns = ["Message","Target"]


In [27]:
#Assigning keys for each dataset
keys = {'Amazon', 'IMDB', 'Yelp'}

#Merging all datasets 
df = pd.concat(frames, keys = keys)

In [28]:
df.shape


(2745, 2)

In [29]:
df.head()

Message  Target
IMDB 0                        Good case, Excellent value.       1
     1                             Great for the jawbone.       1
     2  Tied to charger for conversations lasting more...       0
     3                                  The mic is great.       1
     4  I have to jiggle the plug to get it to line up...       0

In [30]:
#writing it to csv file
df.to_csv("sentimentdataset1.csv")

In [32]:
#data cleaning 
df.columns

Index(['Message', 'Target'], dtype='object')

In [34]:
#checking for null
df.isnull().sum()

Message    0
Target     0
dtype: int64

In [37]:
#spacy
import spacy
#from spacy.lang.en.stop_words import STOP_WORDS
#nlp = spacy.load("en")
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
for word in stopwords:
    if word.is_stop == False and not word.is_punct: